In [1]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.4/422.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s

In [2]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/datas/kdrama.csv")

filter_df = df.drop(["Aired Date","Aired On","Duration","Content Rating","Production companies", "Rank"], axis=1)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  250 non-null    object 
 1   Aired Date            250 non-null    object 
 2   Year of release       250 non-null    int64  
 3   Original Network      250 non-null    object 
 4   Aired On              250 non-null    object 
 5   Number of Episodes    250 non-null    int64  
 6   Duration              250 non-null    object 
 7   Content Rating        245 non-null    object 
 8   Rating                250 non-null    float64
 9   Synopsis              250 non-null    object 
 10  Genre                 250 non-null    object 
 11  Tags                  250 non-null    object 
 12  Director              249 non-null    object 
 13  Screenwriter          249 non-null    object 
 14  Cast                  250 non-null    object 
 15  Production companies  2

In [6]:
filter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                250 non-null    object 
 1   Year of release     250 non-null    int64  
 2   Original Network    250 non-null    object 
 3   Number of Episodes  250 non-null    int64  
 4   Rating              250 non-null    float64
 5   Synopsis            250 non-null    object 
 6   Genre               250 non-null    object 
 7   Tags                250 non-null    object 
 8   Director            249 non-null    object 
 9   Screenwriter        249 non-null    object 
 10  Cast                250 non-null    object 
dtypes: float64(1), int64(2), object(8)
memory usage: 21.6+ KB


In [7]:
import chromadb
client = chromadb.PersistentClient()

collection = client.get_or_create_collection(
    name="k-drama",
    metadata={"hnsw:space": "cosine"}
)


In [8]:
filter_df.iloc[0]

Name                                                     Move to Heaven
Year of release                                                    2021
Original Network                                                Netflix
Number of Episodes                                                   10
Rating                                                              9.2
Synopsis              Geu Roo is a young autistic man. He works for ...
Genre                                            Life,  Drama,  Family 
Tags                  Autism, Uncle-Nephew Relationship, Death, Sava...
Director                                                    Kim Sung Ho
Screenwriter                                               Yoon Ji Ryun
Cast                  Lee Je Hoon, Tang Jun Sang, Hong Seung Hee, Ju...
Name: 0, dtype: object

In [9]:

# 데이터 인덱스
ids = []
# 메타데이터
doc_meta = []
# 벡터로 변환 저장할 텍스트 데이터로 ChromaDB에 Embedding 데이터가 없으면 자동으로 벡터로 변환해서 저장
documents = []

for idx in range(len(filter_df)):
    item = filter_df.iloc[idx]
    id = item['Name'].lower().replace(' ','-')
    document = f"{item['Name']}: {item['Synopsis']} : {str(item['Cast']).strip().lower()} : {str(item['Genre']).strip().lower()}"
    meta = {
        "rating" : item['Rating']
    }

    ids.append(id)
    doc_meta.append(meta)
    documents.append(document)

# DB 저장
collection.add(
    documents=documents,
    metadatas=doc_meta,
    ids=ids
)
# DB 쿼리
collection.query(
    query_texts=["romantic comedy drama"],
    n_results=5,
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:03<00:00, 21.0MiB/s]


{'ids': [['because-this-is-my-first-life',
   'dr.-romantic-2',
   'chicago-typewriter',
   'one-ordinary-day',
   'search:-www']],
 'distances': [[0.5183010697364807,
   0.5386759042739868,
   0.6035799980163574,
   0.615175724029541,
   0.6187994480133057]],
 'metadatas': [[{'rating': 8.5},
   {'rating': 8.7},
   {'rating': 8.7},
   {'rating': 8.5},
   {'rating': 8.4}]],
 'embeddings': None,
 'documents': [['Because This Is My First Life: A house-poor man and homeless woman become housemates in a drama that examines the institution of marriage and the problems that young people face today. : jung so min, lee min ki, esom, park byung eun, kim ga eun, kim min seok : comedy,  romance,  life,  drama',
   'Dr. Romantic 2: A “real doctor” story set in a small, humble hospital called Doldam Hospital. It is a story about people who meet Kim Sa Bu, a genius doctor, and discover “real romance.” : han seok kyu, ahn hyo seop, lee sung kyung, kim joo heon, shin dong wook, so ju yeon : romance,  d